In [1]:
import torch as t
from typing import *
from torch import einsum
from einops import rearrange, reduce, repeat
import gpt_tests
from torch import nn
from torch.nn import Module
from math import sqrt
import bert_sol

## Making the GPT-2 module

In [2]:
class MultiHeadedAttention(Module):
    def __init__(self, hidden_size, num_heads):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_heads = num_heads
        self.head_size = hidden_size/num_heads
        self.attn_lin = nn.Linear(hidden_size, hidden_size*3)
        self.out_lin = nn.Linear(hidden_size, hidden_size)
    
    def forward(self, x):
        product = self.attn_lin(x)
        seq_len = x.shape[1]
        good_format = rearrange(product, 'b n (qkv h p) -> qkv b h n p', qkv = 3, h = self.num_heads)
        queries, keys, values = good_format[0], good_format[1], good_format[2]
        attn_score = t.einsum('bhfp,bhtp -> bhft', keys, queries) / sqrt(self.head_size)
        
        arange = t.arange(seq_len, device=x.device)
        arange_rows = repeat(arange, 'a -> b a', b = seq_len)
        arange_cols = repeat(arange, 'a -> a b', b = seq_len)
        attn_score[:,:,arange_rows < arange_cols] = -1e4
        
        attn_pattn = t.softmax(attn_score, dim=-2)
        # attn_pattn: b h n n; values: b h n p
        out_by_head = t.einsum('bhft,bhfp->bhtp', attn_pattn, values)
        out = rearrange(out_by_head, 'b h t p -> b t (h p)') # b n hidden_size
        return self.out_lin(out)

In [3]:
gpt_tests.test_unidirectional_attn(MultiHeadedAttention)

Congrats! You've passed the test!


In [4]:
class GPT2Block(Module):
    def __init__(self, hidden_size, num_heads, dropout, layer_norm_epsilon):
        super().__init__()
        self.ln1 = nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)
        self.attn = MultiHeadedAttention(hidden_size, num_heads)
        self.ln2 = nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_size, hidden_size*4),
            nn.GELU(),
            nn.Linear(hidden_size*4, hidden_size))
        
    def forward(self, x):
        x1 = self.ln1(x)
        x2 = self.attn(x1) + x
        x3 = self.ln2(x2)
        return self.mlp(x3) + x2
        

In [5]:
gpt_tests.test_gpt_block(GPT2Block)

Congrats! You've passed the test!


In [6]:
from dataclasses import dataclass
from torchtyping import TensorType


@dataclass
class GPT2Output:
    logits: TensorType["batch_size", "vocab_size"]
    final_encoding: TensorType["batch_size", "hidden_size"]

    
class GPT2(Module):
    def __init__(self, num_layers, num_heads, vocab_size, hidden_size,
                 max_position_embeddings, dropout, layer_norm_epsilon):
        super().__init__()
        
        self.token_embedding = nn.Parameter(t.randn(vocab_size, hidden_size))
        self.pos_embedding = nn.Parameter(t.randn(max_position_embeddings, hidden_size))
        
        self.dropout = nn.Dropout(dropout)
        
        self.blocks = nn.Sequential(
            *[GPT2Block(hidden_size, num_heads, dropout, layer_norm_epsilon)
              for _ in range(num_layers)],
            nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)
        )
        
        
    
    def forward(self, x):
        seq_len = x.shape[1]
        embedding = self.token_embedding[x] + self.pos_embedding[:seq_len]
        
        encoding = self.blocks(self.dropout(embedding))
        final_encoding = encoding[:,-1]
        
        logits = t.einsum('vc,bc->bv', self.token_embedding, final_encoding)
        
        return GPT2Output(logits=logits, final_encoding=final_encoding)
    
    
    
    

In [7]:
gpt_tests.test_gpt(GPT2)

Checking logits:
Congrats! You've passed the test!
Checking final encodings:
Congrats! You've passed the test!


## Loading pretrained weights

In [8]:
my_gpt = GPT2(num_layers=12, num_heads=12, vocab_size=50257, hidden_size=768,
                 max_position_embeddings=1024, dropout=0.1, layer_norm_epsilon=1e-5)
pretrained_gpt = gpt_tests.get_pretrained_gpt()

In [9]:
my_gpt = GPT2(num_layers=12, num_heads=12, vocab_size=50257, hidden_size=768,
                 max_position_embeddings=1024, dropout=0.1, layer_norm_epsilon=1e-5)

In [10]:
def string_replace(s):
    s = s.replace("embedding.weight", "embedding")
    s = s.replace("linear1", "mlp.0")
    s = s.replace("linear2", "mlp.2")
    s = s.replace("ln.", "blocks.12.")
    return s

their_dict = pretrained_gpt.state_dict()
for k in list(their_dict.keys()):
    their_dict[string_replace(k)] = their_dict.pop(k)

In [11]:
my_gpt.load_state_dict(their_dict)

<All keys matched successfully>

## Efficient text generation

In [12]:
import transformers
%env TOKENIZERS_PARALLELISM=false

tokenizer = transformers.GPT2Tokenizer.from_pretrained("gpt2")
# print(tokenizer(['Hello, I am a sentence.']))


env: TOKENIZERS_PARALLELISM=false


In [13]:
def feed_gpt(model: nn.Module, text: str, tokenizer, top_k: int = 10):
    input_ids: List[int] = tokenizer(text)["input_ids"]
    logits = model(t.tensor([input_ids], dtype=t.long)).logits
    probs = t.softmax(logits, dim=-1)
    top_logit_idxs = t.argsort(logits, descending=True)[0,:top_k]
    top_logit_words = tokenizer.decode(top_logit_idxs)
    print(top_logit_words)
    print(probs[0,top_logit_idxs])

In [14]:
pretrained_gpt.cpu()
feed_gpt(pretrained_gpt, "Students at the machine learning bootcamp really enjoyed the", tokenizer)

/home/ubuntu/.local/lib/python3.8/site-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


 experience learning opportunity program process course training time work challenge
tensor([0.1915, 0.0389, 0.0338, 0.0290, 0.0174, 0.0165, 0.0162, 0.0151, 0.0129,
        0.0129], grad_fn=<IndexBackward0>)


In [15]:
def feed_gpt_top(model: nn.Module, input_ids: List[int], top_k: int = 10):
    logits = model(t.tensor([input_ids], dtype=t.long)).logits
    probs = t.softmax(logits, dim=-1)
    return t.argsort(logits, descending=True)[0,0]

In [16]:
my_gpt.eval();

In [17]:
start_str = "The machine learning bootcamp started out nicely. But soon, I got an ominous feeling. Shockingly, I discovered"
input_ids = tokenizer(start_str)["input_ids"]
for i in range(100):
    new_token = feed_gpt_top(my_gpt, input_ids, 1)
    print(tokenizer.decode(new_token), end = " ")
    input_ids.append(new_token)

 that  the  machine  learning  boot camp  was  actually  a  very  bad  idea . 
 
 I  was  not  alone .  I  was  also  not  alone  in  my  own  experience .  I  was  also  not  alone  in  my  own  experience . 
 
 I  was  not  alone  in  my  own  experience .  I  was  also  not  alone  in  my  own  experience . 
 
 I  was  not  alone  in  my  own  experience .  I  was  also  not  alone  in  my  own  experience . 
 
 I  was  not  alone  in  my  own  experience .  I  was  also  not  alone  in  my 

In [18]:
class GPT2Modified(Module):
    def __init__(self, num_layers, num_heads, vocab_size, hidden_size,
                 max_position_embeddings, dropout, layer_norm_epsilon):
        super().__init__()
        self.token_embedding = nn.Parameter(t.randn(vocab_size, hidden_size))
        self.pos_embedding = nn.Parameter(t.randn(max_position_embeddings, hidden_size))
        self.dropout = nn.Dropout(dropout)
        self.blocks = nn.Sequential(
            *[GPT2Block(hidden_size, num_heads, dropout, layer_norm_epsilon)
              for _ in range(num_layers)],
            nn.LayerNorm(hidden_size, eps=layer_norm_epsilon)
        )
        
    def forward(self, x):
        seq_len = x.shape[1]
        embedding = self.token_embedding[x] + self.pos_embedding[:seq_len]
        encoding = self.blocks(self.dropout(embedding))
        self.encoding = encoding
        final_encoding = encoding[:,-1]
        logits = t.einsum('vc,bc->bv', self.token_embedding, final_encoding)
        return GPT2Output(logits=logits, final_encoding=final_encoding)

In [19]:
my_gpt_modified = GPT2Modified(num_layers=12, num_heads=12, vocab_size=50257, hidden_size=768,
                 max_position_embeddings=1024, dropout=0.1, layer_norm_epsilon=1e-5)

In [20]:
my_gpt_modified.load_state_dict(their_dict)

<All keys matched successfully>

In [31]:
def create_padded_thing():
    s = 'My life motto: Fortune favors the bold'
    ids = tokenizer(s)["input_ids"]
    return [ids[:i]+[0]*(10-i) for i in range(4,9)]

test_batch = t.tensor(create_padded_thing())

def decode_batch(batch):
    print([tokenizer.decode(batch[i]) for i in range(len(batch))])

In [44]:
my_gpt_modified.eval()
my_gpt_modified(test_batch);
my_gpt_modified.encoding[:,:,:2]

tensor([[[-0.0340, -0.0429],
         [ 0.2075,  0.3059],
         [-0.0342, -0.2292],
         [ 0.3671, -0.0707],
         [-0.0323, -0.3814],
         [-0.0505, -0.2386],
         [ 0.0306, -0.1637],
         [ 0.0751, -0.1712],
         [ 0.0916, -0.1977],
         [ 0.1046, -0.2181]],

        [[-0.0340, -0.0429],
         [ 0.2075,  0.3059],
         [-0.0342, -0.2292],
         [ 0.3671, -0.0707],
         [-0.1335,  0.9349],
         [ 0.3443, -0.1450],
         [ 0.2800, -0.1220],
         [ 0.2185, -0.1960],
         [ 0.2100, -0.1533],
         [ 0.2249, -0.1559]],

        [[-0.0340, -0.0429],
         [ 0.2075,  0.3059],
         [-0.0342, -0.2292],
         [ 0.3671, -0.0707],
         [-0.1335,  0.9349],
         [ 0.1363,  0.0811],
         [ 0.4627, -0.2614],
         [ 0.3536, -0.2568],
         [ 0.1176, -0.2094],
         [ 0.1202, -0.1308]],

        [[-0.0340, -0.0429],
         [ 0.2075,  0.3059],
         [-0.0342, -0.2292],
         [ 0.3671, -0.0707],
        